In [2]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
from collections import defaultdict
from json import JSONDecodeError
from qwikidata.sparql import return_sparql_query_results # python library for working with sparql and linked data from WikiData
import time
from requests.exceptions import ChunkedEncodingError
import math
#from SPARQLWrapper import SPARQLWrapper, JSON # sparql query library
import ssl

# Explore csv

Meglio tenere il corso_codice perchè è la foreign key di DegreeProgrammes e CourseOfStudies (educational activities): serve per unire i due database per facilitare le relazioni successive. 

In [3]:
 # create first dataframe only using the specified columns 
dataDegree = pd.read_csv("DegreeProgrammes.csv", sep=",",
                    usecols=["annoaccademico", "corso_codice", "corso_descrizione", "url", "campus", "sededidattica", "ambiti", "tipologia", "durata", "internazionale", "internazionale_titolo", "accesso"], encoding="utf-8")

# .pickle is a python serialization format for easy and quick read-write, and pandas supports it natively
dataDegree.to_pickle("00_first_Degree.pickle")

# the first table we have looks like this:
dataDegree.head()

,corso_codice,annoaccademico,corso_descrizione,url,campus,sededidattica,ambiti,tipologia,durata,internazionale,internazionale_titolo,accesso
0,659,2018/2019,Giurisprudenza,http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Giurisprudenza,Laurea Magistrale a ciclo unico,5,False,NaN,libero accesso con verifica delle conoscenze
1,660,2018/2019,Giurisprudenza,http://www.unibo.it/it/didattica/corsi-di-stud...,Ravenna,Ravenna,Giurisprudenza,Laurea Magistrale a ciclo unico,5,False,NaN,libero accesso con verifica delle conoscenze
2,881,2018/2019,Architettura,http://www.unibo.it/it/didattica/corsi-di-stud...,Cesena,Cesena,Ingegneria e architettura,Laurea Magistrale a ciclo unico,5,False,NaN,numero chiuso
3,884,2018/2019,Chimica industriale,http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Scienze,Laurea Magistrale,2,True,Titolo multiplo (solo alcuni curricula),libero accesso con verifica dell'adeguatezza d...
4,897,2018/2019,Direzione aziendale,http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Economia e management,Laurea Magistrale,2,True,NaN,numero chiuso


In [4]:
dataCourse = pd.read_csv("CoursesOfStudies.csv", sep=",",
                    usecols=["corso_codice", "materia_codice", "materia_descrizione"], encoding="utf-8")

dataCourse.to_pickle("00_first_Course.pickle")

dataCourse.head()

,corso_codice,materia_codice,materia_descrizione
0,8894,81644,LABORATORY ON RENEWABLE RAW MATERIALS AND CIRC...
1,9222,85238,BAYESIAN INFERENCE
2,8420,87462,HISTORY OF ECONOMIC THOUGHT
3,9222,85199,SURVEY SAMPLING
4,8412,87608,TECNOLOGIA E LEGISLAZIONE FARMACEUTICHE


In [18]:
dataDegree["corso_codice"] = dataDegree["corso_codice"].astype(str)
dataCourse["corso_codice"] = dataCourse["corso_codice"].astype(str)
FinalData = pd.merge(dataDegree, dataCourse, on="corso_codice", how="left")

In [20]:
dataCourse.to_pickle("00_first_Course.pickle")
pd.set_option("display.max_rows", None)
FinalData.head(600)

,corso_codice,annoaccademico,corso_descrizione,url,campus,sededidattica,ambiti,tipologia,durata,internazionale,internazionale_titolo,accesso,materia_codice,materia_descrizione
0,659,2018/2019,Giurisprudenza,http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Giurisprudenza,Laurea Magistrale a ciclo unico,5,False,NaN,libero accesso con verifica delle conoscenze,69910,ANALISI COMPARATA DEI DIRITTI A BASE RELIGIOSA
1,659,2018/2019,Giurisprudenza,http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Giurisprudenza,Laurea Magistrale a ciclo unico,5,False,NaN,libero accesso con verifica delle conoscenze,71815,ATTIVITÀ LIBERE - IDONEITÀ (F)
2,659,2018/2019,Giurisprudenza,http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Giurisprudenza,Laurea Magistrale a ciclo unico,5,False,NaN,libero accesso con verifica delle conoscenze,81809,BIODIRITTO PENALE: STORIA E PRINCIPI
3,659,2018/2019,Giurisprudenza,http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Giurisprudenza,Laurea Magistrale a ciclo unico,5,False,NaN,libero accesso con verifica delle conoscenze,69926,COMPARATIVE CONSTITUTIONALISM
4,659,2018/2019,Giurisprudenza,http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Giurisprudenza,Laurea Magistrale a ciclo unico,5,False,NaN,libero accesso con verifica delle conoscenze,74767,COPYRIGHT LAW
5,659,2018/2019,Giurisprudenza,http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Giurisprudenza,Laurea Magistrale a ciclo unico,5,False,NaN,libero accesso con verifica delle conoscenze,71880,"CRIME, PUNISHMENT AND SOCIETY"
6,659,2018/2019,Giurisprudenza,http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Giurisprudenza,Laurea Magistrale a ciclo unico,5,False,NaN,libero accesso con verifica delle conoscenze,87591,"CRIMINALITA', GENERE E VIOLENZA"
7,659,2018/2019,Giurisprudenza,http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Giurisprudenza,Laurea Magistrale a ciclo unico,5,False,NaN,libero accesso con verifica delle conoscenze,2513,CRIMINOLOGIA
8,659,2018/2019,Giurisprudenza,http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Giurisprudenza,Laurea Magistrale a ciclo unico,5,False,NaN,libero accesso con verifica delle conoscenze,87123,DIDATTICA DEL DIRITTO
9,659,2018/2019,Giurisprudenza,http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Giurisprudenza,Laurea Magistrale a ciclo unico,5,False,NaN,libero accesso con verifica delle conoscenze,46818,DIRITTI FONDAMENTALI
